<a href="https://colab.research.google.com/github/Abdulrahman2k/databricks/blob/master/Apache_Spark_Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install Java, Spark, and Findspark**
This installs Apache Spark 3.0, Java 8, and Findspark, a library that makes it easy for Python to find Spark.

In [19]:
!apt-get install openjdk-8-jdk-headless  -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz    
!tar xf spark-3.0.1-bin-hadoop3.2.tgz   
!pip install -q findspark

!ls

--2020-09-29 10:00:48--  https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224062525 (214M) [application/x-gzip]
Saving to: ‘spark-3.0.1-bin-hadoop3.2.tgz’

spark-3.0.1-bin-had 100%[===================>] 213.68M  19.6MB/s    in 12s     

2020-09-29 10:01:01 (17.8 MB/s) - ‘spark-3.0.1-bin-hadoop3.2.tgz’ saved [224062525/224062525]

BostonHousing.csv  spark-3.0.1-bin-hadoop3.2
sample_data	   spark-3.0.1-bin-hadoop3.2.tgz


# **Set Environment Variables**
Set the locations where Spark and Java are installed.

In [20]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

# **Start a SparkSession**
This will start a local Spark session.

In [21]:
import findspark
findspark.init()

from pyspark import SparkContext 
from pyspark import SparkConf 
from pyspark.sql import SQLContext

conf = SparkConf().setMaster("local[*]").setAppName('pyspark_Linear_Regression') 
sc = SparkContext(conf=conf) 
sqc = SQLContext(sc)

## *Linear Regression Model*

Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. For example, a modeler might want to predict the forecast of the rain based on the humidity ratio. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line.

The goal of this exercise to predict the housing prices by the given features. Let's predict the prices of the Boston Housing dataset by considering MEDV as the output variable and all the other variables as input.

##Read from local Machine , we chose to get from Github

In [4]:
#from google.colab import files
#uploaded = files.upload()

In [22]:
!wget 'https://raw.githubusercontent.com/Abdulrahman2k/databricks/master/BostonHousing.csv'

--2020-09-29 10:02:11--  https://raw.githubusercontent.com/Abdulrahman2k/databricks/master/BostonHousing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35735 (35K) [text/plain]
Saving to: ‘BostonHousing.csv.1’

BostonHousing.csv.1 100%[===================>]  34.90K  --.-KB/s    in 0.008s  

2020-09-29 10:02:11 (4.26 MB/s) - ‘BostonHousing.csv.1’ saved [35735/35735]




Check the dataset is uploaded correctly in the system by the following command

In [23]:
!ls

BostonHousing.csv    sample_data		spark-3.0.1-bin-hadoop3.2.tgz
BostonHousing.csv.1  spark-3.0.1-bin-hadoop3.2


Now that we have uploaded the dataset, we can start analyzing.  For our linear regression model we need to import two modules from Pyspark i.e. Vector Assembler and Linear Regression. Vector Assembler is a transformer that assembles all the features into one vector from multiple columns that contain type double. We could have used StringIndexer if any of our columns contains string values to convert it into numeric values. Luckily, the BostonHousing dataset only contains double values, so we don't need to worry about StringIndexer for now.

In [26]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = sqc.read.csv('BostonHousing.csv',inferSchema=True, header =True)


Notice that we used InferSchema inside read.csv mofule. InferSchema enables us to infer automatically different data types for each column.

Let us print look into the dataset to see the data types of each column:

In [27]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



Next step is to convert all the features from different columns into a single column and let's call this new vector column as 'Attributes' in the outputCol.

In [28]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show()

+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



Here, 'Attributes' are in the input features from all the columns and 'medv' is the target column. Next, we should split the training and testing data according to our dataset (0.8 and 0.2 in this case).

In [29]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])


regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.01096,55.0,2.2...|22.0|28.010660125196154|
|[0.01301,35.0,1.5...|32.7|30.691774456992455|
|[0.0136,75.0,4.0,...|18.9|16.696693688021327|
|[0.01381,80.0,0.4...|50.0|  41.0028316613939|
|[0.01778,95.0,1.4...|32.9|31.595010041997575|
|[0.01951,17.5,1.3...|33.0|23.856636358121293|
|[0.02187,60.0,2.9...|31.1|32.787303344319106|
|[0.02498,0.0,1.89...|16.5|22.494080738981523|
|[0.02985,0.0,2.18...|28.7| 24.59315575619798|
|[0.03306,0.0,5.19...|20.6|22.045602995679328|
|[0.03427,0.0,5.19...|19.5|  19.9886624594851|
|[0.03537,34.0,6.0...|22.0|29.022135013947093|
|[0.03551,25.0,4.8...|22.9|24.746334796518212|
|[0.03578,20.0,3.3...|45.4| 38.12972637064424|
|[0.03584,80.0,3.3...|23.5|30.372913986449028|
|[0.03705,20.0,3.3...|35.4|33.893686880804836|
|[0.04379,80.0,3.3...|19.4| 25.83886329709976|
|[0.04417,70.0,2.2...|24.8| 30.76925911740495|
|[0.04527,0.0


We can also print the coefficient and intercept of the regression model by using the following command:

In [30]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.0911, 0.0471, 0.0243, 2.1211, -16.2409, 4.427, -0.02, -1.3601, 0.2035, -0.0108, -0.9158, 0.0086, -0.3664])
The Intercept of the model is : 30.373802


#Basic Statistical Analysis#

Once we are done with the basic linear regression operation, we can go a bit further and analyze our model statistically by importing RegressionEvaluator module from Pyspark.

In [31]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 7.032
MSE: 49.449
MAE: 4.408
r2: 0.577
